# Flags

> Flags are custom ipython magics, that tell nbdev how to process cells in a notebook.

We test the flags defined in `flags.py` in this notebook.

The nbdev tutorial and other nbdev modules show how these flags are used.

## TODO: move this section to 00_export/tutorial?

Any nbdev import will "import" all flags via `nbdev.__init__.py`. e.g.
- `import nbdev`
- `from nbdev.imports import *`

but if you want the convenience of importing `show_doc` and `notebook2script` as well, use

In [ ]:
from nbdev import *

once imported, we can see that some flags can print warnings

```python
%nbdev_export bad module name
```

<pre>Warning: module_name "bad module name" must not contain whitespace</pre>

and/or feedback

```python
%nbdev_default_export _dummy_flags
```

<pre>
Cells will be exported to nbdev._dummy_flags,
unless a different module is specified after the `%nbdev_export` magic: `%nbdev_export special.module`
</pre>

## We can test magics in ipython ...

In [ ]:
from nbdev.imports import test_eq, IN_IPYTHON
from contextlib import redirect_stdout
import io

if IN_IPYTHON:
    def test_flag_output(s, exp):
        test_eq(s.getvalue().strip(), exp.strip())
    s = io.StringIO()
    with redirect_stdout(s):
        %nbdev_fastai2_test
        test_flag_output(s, '')
        %nbdev_fastai2_test all
        test_flag_output(s, '')
        %nbdev_fastai2_test ALL
        test_flag_output(s, 'UsageError: Invalid option "ALL". Usage `%nbdev_fastai2_test [all]`')

## ... but we need tests to run in normal python too for CI etc

do we need need to run tests in normal python? even if we don't, this kind of testing needs much less code than the above

Note: We don't `del` the functions in `flags.py` so that we can test them here

In [ ]:
def test_flag_output(f, line, exp):
    "Test that `f(line)` prints `exp` to stdout"
    s = io.StringIO()
    with redirect_stdout(s): f(line)
    test_eq(s.getvalue().strip(), exp.strip())

from nbdev.flags import nbdev_default_export
test_flag_output(nbdev_default_export, '', 'UsageError: module_name is missing. Usage `%nbdev_default_export module_name`')
test_flag_output(nbdev_default_export, '  ', 'UsageError: module_name is missing. Usage `%nbdev_default_export module_name`')
test_flag_output(nbdev_default_export, 'bad module name', 'UsageError: module_name "bad module name" must not contain whitespace')
test_flag_output(nbdev_default_export, 'special.module', 
            'Cells will be exported to nbdev.special.module,\n' +
            'unless a different module is specified after an export flag: `%nbdev_export special.module`')

from nbdev.flags import nbdev_export, nbdev_export_and_show, nbdev_export_internal
for f in [nbdev_export, nbdev_export_and_show, nbdev_export_internal]:
    test_flag_output(f, '', '')
    test_flag_output(f, ' ', '')
    test_flag_output(f, 'special.module', '')
    test_flag_output(f, 'bad module name', 'UsageError: module_name "bad module name" must not contain whitespace')

from nbdev.flags import nbdev_hide, nbdev_hide_input, nbdev_hide_output
for f in [nbdev_hide, nbdev_hide_input, nbdev_hide_output]:
    test_flag_output(f, '', '')
    test_flag_output(f, 'any old rubbish', '')

from nbdev.flags import nbdev_default_class_level
for i in range(1,7):
    test_flag_output(nbdev_default_class_level, str(i), '')
test_flag_output(nbdev_default_class_level, '0', 'UsageError: Invalid class level "0". Usage `%nbdev_default_class_level [int between 1 and 6]`')
test_flag_output(nbdev_default_class_level, '7', 'UsageError: Invalid class level "7". Usage `%nbdev_default_class_level [int between 1 and 6]`')
test_flag_output(nbdev_default_class_level, 'not_a_num', 'UsageError: Invalid class level "not_a_num". Usage `%nbdev_default_class_level [int between 1 and 6]`')

from nbdev.flags import nbdev_collapse_input, nbdev_collapse_output
for f in [nbdev_collapse_input, nbdev_collapse_output]:
    test_flag_output(f, '', '')
    test_flag_output(f, ' open ', '')
    test_flag_output(f, 'open', '')
    test_flag_output(f, 'OPEN', f'UsageError: Invalid option "OPEN". Usage `%{f.__name__} [open]`')
    
from nbdev.flags import nbdev_fastai2_test
test_flag_output(nbdev_fastai2_test, '', '')
test_flag_output(nbdev_fastai2_test, ' ', '')
test_flag_output(nbdev_fastai2_test, 'all', '')
test_flag_output(nbdev_fastai2_test, 'All', 'UsageError: Invalid option "All". Usage `%nbdev_fastai2_test [all]`')

# Open questions

## have i missed any flags?

do we need to add any more?

## Is it OK to not support the current comment flags when we switch to magics?

Assuming we write a migration script to replace "comment flags" with "magic flags" in existing projects.

## nbdev_default_export

### Would now be a good time to make the notebook name the module name?
maybe %nbdev_default_export with no module name means we should use the nb name?

## Changes to `__init__.py`

I'm suggesting we add the following to `__init__.py` so that a single `from nbdev import *` gets you the essential magics and the most common nbdev functions (only a few fastai2 nbs don't use show_doc and notebook2script).

``` python
if IN_IPYTHON:
    from .flags import *
    from .showdoc import show_doc
    from .export import notebook2script
```

even if we don't include show_doc and notebook2script in \_\_init__.py, we can have a 1st cell that looks like;
``` python
from nbdev import *
%nbdev_default_export export2html
%nbdev_default_class_level 3
```

## how about we consolidate to just 2 collapse flags ...

that are closed by default - but have a single option to make the collapse element open by default

```python
%nbdev_collapse_input
%nbdev_collapse_input open

%nbdev_collapse_output
%nbdev_collapse_output open
```

we currently have collapse, collapse_show, collapse-show, collapse_hide, collapse-hide, collapse_output, collapse-output (and the docs refer to collapse_open <- might be a mistake?)

In [ ]:
%nbdev_hide
notebook2script()

Converted 00_export.ipynb.
Converted 01_sync.ipynb.
Converted 02_showdoc.ipynb.
Converted 03_export2html.ipynb.
Converted 04_test.ipynb.
Converted 05_merge.ipynb.
Converted 06_cli.ipynb.
Converted 07_clean.ipynb.
Converted 08_flags.ipynb.
Converted 99_search.ipynb.
Converted index.ipynb.
Converted tutorial.ipynb.
